In [10]:
!pip3 install tensorflow_text>=2.0.0rc0

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import os



import os
import tensorflow as tf
import tensorflow_hub as hub

#the commented code is to experiment with caching the module
# os.environ['TFHUB_CACHE_DIR'] = '/Users/zakikidane/Documents/Advice_AI/Q_and_A_solution' #Any folder that you can access
# elmo_model = hub.Module('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=True)
# # Load the model
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [3]:
def loadFile(filepath):
    with open(filepath, "r") as f: lines = f.readlines()
    lines = [line[:-2] for line in lines] # removes the newlines at the end
    return lines
    


In [14]:
def ask(query, lines):
  # Encode the question (or more aptly an array of the single function)
    advice = tf.constant(lines)
    advice_embeddings = module.signatures['response_encoder'](
    input=advice,context=advice) 
    question_embeddings = module.signatures['question_encoder'](tf.constant([query]))

    # Get an array of floats for how "good" each adivce is for this query
    # We can do this via dot product because math
    # (and this is secretly a similarity problem)
    weights = np.inner(question_embeddings['outputs'],
                     advice_embeddings['outputs'])[0] # [0] because just one query

  # Returns a sorted tuple list with advice and
  # how "good" they are for the given query
    return sorted([(lines[i], weights[i]) for i in range(len(lines))], key=lambda x: -x[1])

def testFiles(query, filepath):
    lines = loadFile(filepath)
    n = 3
    results = list(ask(query, lines)[:n])
    #print('For advice file of {} the score is {} .'.format(filepath, sum(r[1] for r in results)))
    val = sum(r[1] for r in results)
    return val

In [15]:
files = ["../data/advice.txt", "../data/heyfromthefuture/filtered.txt"]
queries = ["im overweight", "my girlfriend left me", "I Am sadd", "I am Happo", "why is life so hard","How do I pay off my debt?","I don't want children"]
scores = {files[0]:[], files[1]:[]}

for file in files: 
    for query in queries:
        score = testFiles(query, file)
        print('score is', score)
        scores[file].append(score)
         



score is 0.5704040080308914
score is 0.6048931777477264
score is 0.28732863813638687
score is 0.1236298717558384
score is 0.9196281731128693
score is 1.0333175957202911
score is 0.6869828104972839
score is 0.6787922084331512
score is 0.9357615411281586
score is 0.4119967073202133
score is 0.1684480793774128
score is 0.9635390043258667
score is 1.528080403804779
score is 0.8309200257062912


In [16]:
print(scores)

{'../data/advice.txt': [0.5704040080308914, 0.6048931777477264, 0.28732863813638687, 0.1236298717558384, 0.9196281731128693, 1.0333175957202911, 0.6869828104972839], '../data/heyfromthefuture/filtered.txt': [0.6787922084331512, 0.9357615411281586, 0.4119967073202133, 0.1684480793774128, 0.9635390043258667, 1.528080403804779, 0.8309200257062912]}
